In [1]:
import numpy as np
import pandas as pd
import data_als
import data_prepro

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable 

import torch.optim as optim

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train,test = data_prepro.loadData('datasets/data_tr_city.csv','datasets/data_ts_city.csv',trainYearRange1=2019,trainYearRange2=2020)

In [4]:
train = data_prepro.outlierDataToNan(train,low=True,high=False,thres=10.0)
# train = data_prepro.fillZero(train)
train = data_prepro.fillnaBehind(train)
train = data_prepro.fillprevValue(train)


Int64Index([17558, 21595, 25626], dtype='int64')
index                  0
datetime               0
구미 혁신도시배수지 유출유량 적산차    2
year                   0
month                  0
dtype: int64
